In [2]:
import os
from pathlib import Path
import requests
import pickle
import gzip

pj_root = os.getcwd()
Data = Path(r'data')
FileName = 't10k-labels-idx1-ubyte.gz'
with gzip.open((Data / FileName).as_posix(), "rb") as f:
    ((x_train, y_train), (x_valid, y_valid), (x_test, y_test)) = pickle.load(f)

UnpicklingError: invalid load key, '\x00'.

In [6]:
import numpy as np
def read_image(file_path):
    """读取MNIST图片

    Args:
        file_path (str): 图片文件位置

    Returns:
        list: 图片列表
    """
    with open(file_path,'rb') as f:
        file = f.read()
        img_num = int.from_bytes(file[4:8],byteorder='big') #图片数量
        img_h = int.from_bytes(file[8:12],byteorder='big') #图片h
        img_w = int.from_bytes(file[12:16],byteorder='big') #图片w
        img_data = []
        file = file[16:]
        data_len = img_h*img_w

        for i in range(img_num):
            data = [item/255 for item in file[i*data_len:(i+1)*data_len]]
            img_data.append(np.array(data).reshape(img_h,img_w))

        return img_data

def read_label(file_path):
    with open(file_path,'rb') as f:
        file = f.read()
        label_num = int.from_bytes(file[4:8],byteorder='big') #label的数量
        file = file[8:]
        label_data = []
        for i in range(label_num):
            label_data.append(file[i])
        return label_data


train_img  = read_image("data/train-images.idx3-ubyte")
train_label = read_label("data/train-labels.idx1-ubyte")

In [ ]:
from sklearn.model_selection import train_test_split
train_img,valid_img,train_label,valid_label = train_test_split(train_img,train_label,test_size=0.2,shuffle=True)

In [ ]:
import torch

train_loader = torch.utils.data.DataLoader(train_img)

In [7]:
import torchvision
import torch
import torchvision.transforms as transforms

# Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.Compose([
                                               # transforms.Resize((32, 32)),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=(0.1307,), std=(0.3081,))]),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.Compose([
                                              # transforms.Resize((32, 32)),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean=(0.1325,), std=(0.3105,))]),
                                          download=True)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True,
                                           num_workers=4)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=64,
                                           shuffle=True,
                                           num_workers=4)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=4)